### Heuristics for active learning for regression

We consider the linear/nonlinear regression settings where we would like to learn a regressor h, which is a function that maps some feature space $\mathcal{X} \in \mathbb{R}^d$ to a probability distribution over a label space $\mathcal{Y}:$
$$h: \mathcal{X} \rightarrow p(\mathcal{Y})$$

The goal for active learning to come up with a rule $s(x; h)$ that gives each unlabeled example a score based only on their feature vector $x$ and the current regressor $h$. Recall that the regressor produces $p(\mathcal{Y})$, a probability estimate for each label. We use these probability estimates from the regressor over the unlabeled examples to calculate the scores:
$$s:p(\mathcal{Y})→\mathbb{R}$$

The value of $s(x; h)$ indicates the informativeness of example $x$, where bigger is better. We would then label the example with the largest value of $s(x; h)$. This will be our active learning rule r:
$$r(U;h) = {argmax}_{x \in U} s(x;h)$$
where U is the unlabeled set $U \subset \mathcal{X}$.  

Then the idea is that if $p(\mathcal{Y})$ can provide not only a probablity for a specific x, but also the variance/error/uncertainty for that x, then we can utilise the uncertainty to indicate the informativeness of example $x$ directly. 


#### Heuristic 1: 

Consider the Bayesian linear regression setting, we obtain the (posterior) predictive distribution (assume Gaussian prior and likelihood):

$$p(y_\ast | \mathcal{X,Y}, x_\ast) = \int p(y_\ast | x_ast, \theta) p(\theta | \mathcal{X,Y}) d\theta = \mathcal{N} (y_\ast | \mu_\ast, \sigma_\ast)$$

where $x_\ast$ is test input and $y_\ast$ is corresponding prediction. $\sigma_\ast$ reflects the posterior uncertainty associated with the parameter $\theta$ and data noise. The predictive mean $\mu_\ast$ coincides with the MAP estimate.

Then the active learning rule r could be:

$$r_{BLR1} (U; h) = {argmax}_{x \in U} \sigma_{\ast} \tag{1}$$

Further more, say if we care about the unlabeled instances with prediction values around a certain value $m$. We can choose the instances with predictive mean close to $m$ but with relative large uncertainty.

$$r_{BLR2} (U; h) = {argmax}_{x \in U} \{- (\mu_\ast - m)^2 + \beta \sigma_{\ast}\} \tag{2}$$

where $\beta$ indicates how much we care about uncertainty.

#### Heuristic 2:

Consider the Gaussian Process regression setting, which can be regarded as a special case for Bayesian linear regression setting (with the prior as Gaussian process). According to the assumption in GP modelling, where our data can be represented as a sample from a multivariate Gaussian distribution, we have 

$$
 \left[
 \begin{matrix}
   y  \\
   y_\ast  \\
  \end{matrix}
  \right] 
\sim
\mathcal{N}(0, 
\left[
\begin{matrix}
   K, K_\ast^T  \\
   K_\ast, K_{\ast\ast}  \\
  \end{matrix}
  \right] )
  \tag{3}
$$

where K is the covariance matrix for observed data pair (x, x'), $K_\ast  = [k(x_\ast, x_1) \quad k(x_\ast, x_2) \quad ... \quad k(x_\ast, x_n)]$ and $K_{\ast\ast} = k(x_\ast, x_\ast)$

And we will have a gaussian predictive distribution for the condition probability $p(y_\ast | y)$:
$$p(y_\ast | y) = \mathcal{N}(\mu_\ast, \sigma_\ast)$$

The active learning rule is same as Equation (1) and (2).

#### Heuristic 3:

Consider quantile regression setting, for $x_\ast$ we can have the median prediction $u_\ast$ (i.e. 50%), along with the upper bound as q-quantile prediction $q_\ast$(e.g. 80%) and the lower bound as p-quantile prediction $p_\ast$(e.g. 20%). Then the active learning rule is:

$$r_{QR} (U; h) = {argmax}_{x \in U} \{q_{\ast} - p_\ast \tag{4}\}$$

Again if we care about the prediction value around a certain value m, 

$$r_{QR} (U; h) = {argmax}_{x \in U} \{- (u_\ast - m)^2 + \beta (q_{\ast} - p_\ast)\} \tag{5}$$

The above heuristics can be easily extended to the classification setting, as long as the predictive probablity distribution $p(\mathcal{Y})$ can provide the uncertainty for the unlabelled instances.

Since linear regression problem can be well generalized to matrix factorization problems, these heuristics can also be used for tensors (knowledge population).

### Batch Active learning

We already get the informativeness score s(x; h) for all unlabeled instances x. Now we consider what if we want to recommend n instances (n > 1) for one iteration, what will be the active learning rule $r_n(U; h)? We can simply use the top n score as recommendation. However, the top n instances may provide similar information as each other, say we can inferring the label as the one instance by knowing the label of another instance.

#### Kmeans++ 

For one batch we want to recommend m instances to be labelled, then we run kmeans++ algorithm on the top n informative instances and pick m instances which are furthest to each other (the distance should depend on the features).

#### Gaussian process

As indicated in Equation (3), the similarity for all data points have already been presented by the covirance matrix (kernel function). 

The first option is to use the approaches in Kmeans++ with the distance between data points as the k(x,x'). We first pick the top 1 instance $x_{1*}$ and then find the instance $x_{t*}$ which has the smallest similarity with $x_{1* :(t-1)*}$ (t >= 2). 

The second option is:   
From all unlabeled set U, we want to recommend m data points. First we pick the top n data points (len(U) > n > m, e.g. n = 30 and m = 10). 
1. From s(x; h) we get the $\textbf{Uncertainty Rank}$ for n data points.
1. Get the $n \times n$ matrix $K_n$ with entries as the similarities k(x, x'), then marginalise each row in {K_n} and sort the summation to get the $\textbf{Distance Rank}$.
3. sort the sum of Uncertainty Rank and Distance Rank, get the total rank.

Example:

|   | 1   | 2   | 3   | Sum | Distance Rank | Uncertainty Rank | Total rank |
|---|-----|-----|-----|-----|---------------|------------------|------------|
| 1 | 1   | 0.5 | 0.3 | 1.8 | 1             | 3                | 2          |
| 2 | 0.5 | 1   | 0.2 | 1.7 | 2             | 1                | 1          |
| 3 | 0.3 | 0.2 | 1   | 1.5 | 3             | 2                | 3          |

Or instead of mapping the similarity and uncertainty into ranking, we can normalize both distance sum and uncertainty to [0,1] (represented as $\mathbf{d}$ and $\mathbf{u}$):

$$r(U;h) = {argmax}_{x \in U} \{\mathbf{u} - \beta \mathbf{d}\}$$

Then get the top m based that rule.